In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

In [2]:
housing = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

###  Passing the features through deep and wide channels
#### This allows the neural network to learn both deep patterns (using deep path) and simple rules (through the short path)
#### In contrast, a regular MLP forces all the data to flow through the full stack of layers; thus, simple patterns in the data may end up being distorted by this sequence of transformations

In [4]:
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden = keras.layers.Dense(30, activation="relu")(input_)
hidden = keras.layers.Dense(30, activation="relu")(hidden)
concat = keras.layers.Concatenate()([input_, hidden])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_], outputs=[output])

model.compile(loss="mean_squared_error", optimizer="sgd")

history = model.fit(X_train, y_train, epochs=20, validation_split=0.2)

mse_test = model.evaluate(X_test, y_test)

X_new = X_test[:3] #pretend these are new instances to do some testing
y_pred = model.predict(X_new)
print(y_test[:3], y_pred)

Train on 12384 samples, validate on 3096 samples
Epoch 1/20
12384/12384 [==============================] - 0s 40us/sample - loss: 1.6435 - val_loss: 0.7773
Epoch 2/20
12384/12384 [==============================] - 0s 27us/sample - loss: 0.7673 - val_loss: 0.7218
Epoch 3/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.8438 - val_loss: 0.6746
Epoch 4/20
12384/12384 [==============================] - 0s 24us/sample - loss: 0.7026 - val_loss: 0.6325
Epoch 5/20
12384/12384 [==============================] - 0s 25us/sample - loss: 0.7057 - val_loss: 0.5955
Epoch 6/20
12384/12384 [==============================] - 0s 25us/sample - loss: 0.5956 - val_loss: 0.5536
Epoch 7/20
12384/12384 [==============================] - 0s 24us/sample - loss: 0.5734 - val_loss: 0.5444
Epoch 8/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.5333 - val_loss: 0.5221
Epoch 9/20
12384/12384 [==============================] - 0s 25us/sample - loss: 0.5288 - val_l

### Handling multiple inputs - one set of features through the deep channel and the other through wide channel (possibly overlapping set of features)

In [5]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden = keras.layers.Dense(30, activation="relu")(input_B)
hidden = keras.layers.Dense(30, activation="relu")(hidden)
concat = keras.layers.Concatenate()([input_A, hidden])
output = keras.layers.Dense(1)(concat)
model = keras.Model(inputs=[input_A, input_B], outputs=[output])

model.compile(loss="mean_squared_error", optimizer="sgd")

history = model.fit((X_train[:, :5], X_train[:, 2:]), y_train, epochs=20, validation_split=0.2)

mse_test = model.evaluate((X_test[:, :5], X_test[:, 2:]), y_test)

X_new = X_test[:3] #pretend these are new instances to do some testing
y_pred = model.predict((X_new[:, :5], X_new[:, 2:]))
print(y_test[:3], y_pred)

Train on 12384 samples, validate on 3096 samples
Epoch 1/20
12384/12384 [==============================] - 0s 37us/sample - loss: 1.6736 - val_loss: 0.8682
Epoch 2/20
12384/12384 [==============================] - 0s 28us/sample - loss: 0.7824 - val_loss: 0.7403
Epoch 3/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.6973 - val_loss: 0.6808
Epoch 4/20
12384/12384 [==============================] - 0s 25us/sample - loss: 0.6498 - val_loss: 0.6376
Epoch 5/20
12384/12384 [==============================] - 0s 25us/sample - loss: 0.6152 - val_loss: 0.6063
Epoch 6/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.5887 - val_loss: 0.5803
Epoch 7/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.5671 - val_loss: 0.5579
Epoch 8/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.5470 - val_loss: 0.5387
Epoch 9/20
12384/12384 [==============================] - 0s 26us/sample - loss: 0.5335 - val_l

### Handling multiple outputs; In this case to add an auxillary output for regularization
#### Each output needs its own loss function. So, we pass a list of losses
#### By default, keras will compute all these losses and simply add them up to get the final loss used for training. We care musch more about the main output then about the aux output. So, we give more weight to output's loss than aux output's loss

In [6]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden = keras.layers.Dense(30, activation="relu")(input_B)
hidden = keras.layers.Dense(30, activation="relu")(hidden)
concat = keras.layers.Concatenate()([input_A, hidden])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden)
model = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

model.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

history = model.fit((X_train[:, :5], X_train[:, 2:]), (y_train, y_train), epochs=20, validation_split=0.2)

mse_test = model.evaluate((X_test[:, :5], X_test[:, 2:]), (y_test, y_test))

X_new = X_test[:3] #pretend these are new instances to do some testing
y_pred = model.predict((X_new[:, :5], X_new[:, 2:]))
print(y_test[:3], y_pred)

Train on 12384 samples, validate on 3096 samples
Epoch 1/20
12384/12384 [==============================] - 1s 55us/sample - loss: 2.7194 - main_output_loss: 2.4928 - aux_output_loss: 4.7591 - val_loss: 1.3870 - val_main_output_loss: 1.1672 - val_aux_output_loss: 3.3646
Epoch 2/20
12384/12384 [==============================] - 0s 31us/sample - loss: 1.1763 - main_output_loss: 0.9984 - aux_output_loss: 2.7778 - val_loss: 0.9468 - val_main_output_loss: 0.8121 - val_aux_output_loss: 2.1591
Epoch 3/20
12384/12384 [==============================] - 0s 29us/sample - loss: 0.8445 - main_output_loss: 0.7178 - aux_output_loss: 1.9853 - val_loss: 0.7699 - val_main_output_loss: 0.6691 - val_aux_output_loss: 1.6772
Epoch 4/20
12384/12384 [==============================] - 0s 29us/sample - loss: 0.7407 - main_output_loss: 0.6377 - aux_output_loss: 1.6684 - val_loss: 0.7079 - val_main_output_loss: 0.6212 - val_aux_output_loss: 1.4880
Epoch 5/20
12384/12384 [==============================] - 0s 29us/s